In [87]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from mp3id import mp3ID
from musicBase import musicBase
from musicFinder import musicFinder
from musicPath import pathBasics
from timeUtils import clock, elapsed
from discogsBase import discogs
from multiArtist import multiArtist
from time import sleep
from pandas import DataFrame, Series
from discogsBase import discogs
from ioUtils import getFile
from searchUtils import findDirs
from fileUtils import getDirBasics
from unicodedata import normalize
from fsUtils import moveDir, setDir
from masterdb import getArtistAlbumsDB, discConv
## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-24 07:24:12.244173


In [88]:
disc = discogs()

Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


In [89]:
print("Getting Master Artist DB File")
saveFilename = disc.getMasterSlimArtistDiscogsDBFilename()
print(saveFilename)
discdf = getFile(ifile=saveFilename, debug=False)
artists = list(discdf["Artist"])
print(len(artists))
discdf.head()
# 589630
# 610522
# 636524
# 669087
# 712336

Getting Master Artist DB File
/Volumes/Music/Discog/db/MasterSlimArtistDB.p
712336


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


In [90]:
artistAlbumsDB = getArtistAlbumsDB(disc)

Current Time is Fri Jan 24, 2020 07:24:24 for 
=================================== Creating Artist Albums DB ===================================
Loading ArtistID Data
Loading data from /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
  --> This file is 188.6MB.
Loading /Volumes/Music/Discog/db/ArtistIDToAlbumNames.p
Creating Pandas DataFrame for 712336 Artists
	Shape --> (712336, 1)
DataFrame Shape is (712336, 1)
Current Time is Fri Jan 24, 2020 07:27:25 for Done with 
=================================== Creating Artist Albums DB ===================================
Process [{0}] took 3.0 minutes.


In [91]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
from difflib import SequenceMatcher

def getRowByIndex(pdf, idx):
    return pdf.loc[idx]

In [92]:
artistIDToName = discdf["DiscArtist"].to_dict()
artistNameToID = {}
print("Found {0} ID -> Name entries".format(len(artistIDToName)))
for artistID,artistName in artistIDToName.items():
    if artistNameToID.get(artistName) is None:
        artistNameToID[artistName] = []
    artistNameToID[artistName].append(artistID)
print("Found {0} Name -> ID entries".format(len(artistNameToID)))

Found 712336 ID -> Name entries
Found 654724 Name -> ID entries


In [93]:
#artistDB = disc.getArtistIDToNameData()
#artistNameToID = {v: k for k,v in artistDB.items()}
mulArts  = multiArtist(cutoff=0.9, discdata=artistNameToID, exact=False)

In [94]:
try:
    myMusicNameIDMap = getFile("musicDiscogsMap.p")
    print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))
except:
    myMusicNameIDMap = {}
    print("Could not load music <-> discogs map")
       
myMusicNameIDMap["Mayday!"] == ['2156710', '¡Mayday!']

Found 4698 music <-> discogs maps


True

# Fix Discogs Mapping

In [95]:
def fixDiscogsMapping(myMusicNameIDMap):
    musicNameIDMap = {}
    for artistName,v in myMusicNameIDMap.items():
        if isinstance(v, list):
            musicNameIDMap[artistName] = v
            #print(artistName,v)
            continue

        mdata = getMusicData("Name", v)
        if isinstance(mdata, DataFrame):
            if mdata.shape[0] == 1:
                #print(artistName,[mdata.index[0], mdata["Name"].values[0]])
                musicNameIDMap[artistName] = [mdata.index[0], mdata["Name"].values[0]]
            else:
                raise ValueError("No idea about {0}".format(mdata))
        else:
            print(artistName,'\t',v)
            
    return musicNameIDMap
        
#myMusicNameIDMap = musicNameIDMap
#from ioUtils import saveFile
#saveFile(ifile="musicDiscogsMap.p", idata=musicNameIDMap, debug=True)

In [96]:
myMusicNameIDMap = fixDiscogsMapping(myMusicNameIDMap)

In [ ]:
if myMusicNameIDMap["Mayday!"] == ['2156710', '¡Mayday!']:
    saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)
    saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

In [ ]:
manualMatches = {}

if len(manualMatches) > 0:
    myMusicNameIDMap.update(manualMatches) 

print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

# Find Music and Match Artists

In [ ]:
musicNameIDMap = {}
multiMap       = {}

discogMediaNames = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
myMediaNames     = ['Random', 'Todo', 'Match', 'Title', 'Singles']

for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        if myMusicNameIDMap.get(artistName) is not None:
            continue

        mdata = getMusicData("DiscArtist", artistName)
        print(artistName,type(mdata))
        if not isinstance(mdata, DataFrame):
            matches = mulArts.getArtistNames(artistName)
            if not all(matches.values()):
                print("{0: <10}{1: <50}{2}".format("?", artistName, matches))
                continue
            else:
                print("{0: <10}{1: <50}{2}".format("MIX", artistName, matches))
                srcDir = dirval
                dstDir = setDir("/Volumes/Music/Multi", artistName)
                moveDir(srcDir, dstDir)                
        else:
            print('\t',mdata.shape)
            if mdata.shape[0] == 1:
                musicNameIDMap[artistName] = [mdata.index, mdata["Name"].values[0]]
            else:
                print("{0: <10}{1: <50}{2}".format("", artistName, ""))
                albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in discogMediaNames+myMediaNames]
                if len(albums) == 0:
                    print("\tNo Albums\n")
                    srcDir = dirval
                    dstDir = setDir("/Volumes/Music/Random", artistName)
                    moveDir(srcDir, dstDir)
                else:
                    multiMap[artistName] = {"DB": mdata, "Albums": albums}
                    continue
                    for album in albums:
                        print("\t",album)
                    print("")
                print("\tDiscogs Data:")
                print(mdata)
                print("\n\n")

# Manual Entries

In [ ]:
newManual = {}

In [ ]:
#mdata = getMusicData("Name", "Dababy")
#newManual[mdata["DiscArtist"].values[0]] = [mdata.index, mdata["Name"].values[0]]

In [ ]:
newManual

In [ ]:
if len(newManual) > 0:
    try:
        myMusicNameIDMap.update({k: [v[0][0], v[1].values[0]] for k,v in newManual.items()})
    except:
        try:
            myMusicNameIDMap.update({k: [v[0][0], v[1]] for k,v in newManual.items()})
        except:
            raise ValueError("Not sure about {0}".format(newManual))

## Find MultiMap -> Match Entry

In [ ]:
ratVal = 0.6
if len(myMusicNameIDMap) > 0:
    ratVal = 0.3
artistIDMap = {}
for artistName,artistSlimData in multiMap.items():
    print("\n","="*50)
    print("ArtistName: {0}".format(artistName))
    print("   Albums: {0}".format(len(artistSlimData["Albums"])))

    myAlbumNames = []
    for album in artistSlimData["Albums"]:
        myAlbumName = album.split("/{0}/".format(artistName))[-1]
        #print("     {0: <15}{1: <10}{2}".format("", "", myAlbumName))
        myAlbumNames.append(myAlbumName)
        
    for idx,row in artistSlimData["DB"].iterrows():
        artistAlbumsData = getRowByIndex(artistAlbumsDB, idx)
        artistAlbums     = artistAlbumsData["Albums"]
        print("   Match: {0}  [{1}]".format(row["DiscArtist"], row["Name"]))
        if isinstance(artistAlbums, dict):
            for albumType, albumTypeData in artistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    for myAlbumName in myAlbumNames:
                        s = SequenceMatcher(None, myAlbumName, dbAlbumName)
                        ratio = s.ratio()
                        if ratio > 0.6:
                            print("     {0: <15}{1: <10}{2: <8}{3}".format(albumType, albumID, round(ratio,2), dbAlbumName))
                            artistIDMap[artistName] = [idx, row["Name"]]

## Found Multi Artist

In [ ]:
for artistName,artistData in artistIDMap.items():
    print(artistName,' ==> ',artistData)

In [ ]:
if len(artistIDMap) > 0:
    myMusicNameIDMap.update(artistIDMap)

## Set Single Artist

In [ ]:
for artistName,artistData in musicNameIDMap.items():
    print(artistName,' ==> ',artistData)

In [ ]:
print("Found {0} entries".format(len(musicNameIDMap)))
if len(musicNameIDMap) > 0:
    try:
        myMusicNameIDMap.update({k: [v[0][0], v[1].values[0]] for k,v in musicNameIDMap.items()})
    except:
        try:
            myMusicNameIDMap.update({k: [v[0][0], v[1]] for k,v in musicNameIDMap.items()})
        except:
            raise ValueError("Not sure about {0}".format(musicNameIDMap))
print("Found {0} entries".format(len(musicNameIDMap)))

# Save Everything

In [ ]:
from ioUtils import saveFile
print("Saving {0} entries".format(len(myMusicNameIDMap)))
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)

In [ ]:
from ioUtils import saveFile
print("Saving {0} entries".format(len(myMusicNameIDMap)))
saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

In [18]:
myMusicNameIDMap = getFile("musicDiscogsMap.p")
print("Found {0} music <-> discogs maps".format(len(myMusicNameIDMap)))

Found 4698 music <-> discogs maps


In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.p", idata=myMusicNameIDMap, debug=True)
from ioUtils import saveFile
saveFile(ifile="musicDiscogsMap.yaml", idata=myMusicNameIDMap, debug=True)

# Find Music and Match Albums

In [19]:
myMusicAlbumIDMap = getFile("musicDiscogsAlbumMap.p")
#myMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumIDMap)))

Found 1848 music <-> discogs albums maps


In [20]:
skipMusicAlbumIDMap = getFile(ifile="skipMusicDiscogsAlbumMap.yaml")
#skipMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))

Found 169 music <-> discogs albums maps


## Johnny Cash

In [103]:
discogsArtist            = myMusicNameIDMap["James Brown"]
idx                      = discogsArtist[0]
discogsArtistName        = discogsArtist[1]
discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]

In [105]:
for k,v in discogsArtistAlbums.items():
    for code,name in v.items():
        #print("{0: <15}{1: <12}{2}".format(k,code,name))
        continue

# Show and Find Music <-> Discog Matches

In [85]:
ratioCut = 0.67
keepSkips = False

In [97]:
def showArtist(artistName, discogsArtist):
    print("\t{0: <30} ---> {1}".format(artistName, discogsArtist))
    
def showAlbum(myAlbumName):
    print("\t\tMy Album: {0}".format(myAlbumName))
    
def showMatch(myAlbumName, albumData):
    print("\t\t   Match: {0}".format(albumData))

musicAlbumIDMap = {}
discogMediaNames = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
myMediaNames     = ['Random', 'Todo', 'Match', 'Title', 'Singles', 'Album', 'Unknown', 'Bootleg', 'Mix', 'Tribute']
ignoreDirs       = list(set(discogMediaNames+myMediaNames))
nSkips = len(skipMusicAlbumIDMap)



for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        discogsArtist = myMusicNameIDMap[artistName]
        
        
        idx                      = discogsArtist[0]
        discogsArtistName        = discogsArtist[1]
        discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
        discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
        
        albums = [x for x in findDirs(dirval) if getDirBasics(x)[-1] not in ignoreDirs]        
        myAlbumNames = [album.split("/{0}/".format(artistName))[-1] for album in albums]
        
        for myAlbumName in myAlbumNames:
            myAlbumName = normalize('NFC', myAlbumName)
            if myMusicAlbumIDMap.get(artistName) is not None:
                if myMusicAlbumIDMap[artistName].get(myAlbumName) is not None:
                    #print("\t\tMy Album: {0} ---> {1}".format(myAlbumName, myMusicAlbumIDMap[artistName][myAlbumName]))
                    continue
            if skipMusicAlbumIDMap.get(artistName) is not None:
                if skipMusicAlbumIDMap[artistName].get(myAlbumName) is not None:
                    #print("\t\tMy Album: {0} ---> {1}".format(myAlbumName, myMusicAlbumIDMap[artistName][myAlbumName]))
                    continue

            maxRatio  = None
            albumData = None
            for albumType, albumTypeData in discogsArtistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    convDiscogsAlbumName = discConv(dbAlbumName)
                    s = SequenceMatcher(None, myAlbumName, convDiscogsAlbumName)
                    ratio = round(s.ratio(), 3)
                    if ratio > ratioCut:
                        if maxRatio is None:
                            maxRatio = ratio
                        else:
                            if ratio < maxRatio:
                                continue
                            maxRatio  = ratio
                            albumData = [albumID, dbAlbumName]
                            
            if albumData is not None:
                if musicAlbumIDMap.get(artistName) is None:
                    musicAlbumIDMap[artistName] = {}
                    showArtist(artistName, discogsArtist)
                showAlbum(myAlbumName)
                musicAlbumIDMap[artistName][myAlbumName] = albumData
                showMatch(myAlbumName, albumData)
                        
            if keepSkips:
                if skipMusicAlbumIDMap.get(artistName) is None:
                    skipMusicAlbumIDMap[artistName] = {}
                #showAlbum(myAlbumName)
                skipMusicAlbumIDMap[artistName][myAlbumName] = [albumID, dbAlbumName]



/Users/tgadfort/matched/A
	AC-DC                          ---> ['84752', 'AC/DC']
		My Album: '74 Jailbreak
		   Match: ['8460', "'74 Jailbreak"]
		My Album: Back In Black
		   Match: ['324463', 'Back In Black']
		My Album: Ballbreaker
		   Match: ['3654907', 'Ballbreaker']
		My Album: Dirty Deeds Done Dirt Cheap
		   Match: ['469928', 'Dirty Deeds Done Dirt Cheap']
		My Album: Dirty Deeds Done Dirt Cheap (Live)
		   Match: ['201450', 'Dirty Deeds Done Dirt Cheap (Live)']
		My Album: Flick Of The Switch
		   Match: ['545065', 'Flick Of The Switch']
		My Album: Fly On The Wall
		   Match: ['291661', 'Fly On The Wall']
		My Album: For Those About To Rock (We Salute You)
		   Match: ['8663', 'For Those About To Rock (We Salute You)']
		My Album: High Voltage
		   Match: ['8437', 'High Voltage']
		My Album: Highway To Hell
		   Match: ['8659', 'Highway To Hell']
		My Album: Let There Be Rock
		   Match: ['241625', 'Let There Be Rock']
		My Album: Stiff Upper Lip
		   Match: ['167008', 'Sti

## Check For Skips

In [56]:
#skipMusicAlbumIDMap

In [55]:
print("Previous {0} music <-> discogs albums maps".format(nSkips))
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))
saveFile(ifile="skipMusicDiscogsAlbumMap.yaml", idata=skipMusicAlbumIDMap, debug=True)


Previous 0 music <-> discogs albums maps
Found 1163 music <-> discogs albums maps
Saving data to skipMusicDiscogsAlbumMap.yaml
Saved data to skipMusicDiscogsAlbumMap.yaml
  --> This file is 230.4kB.


In [63]:
skipMusicAlbumIDMap = getFile(ifile="skipMusicDiscogsAlbumMap.yaml")
skipMusicAlbumIDMap = {}
print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))

Found 0 music <-> discogs albums maps


## Check For Match

In [98]:
from ioUtils import saveFile
saveFile(ifile="newMusicDiscogsAlbumMap.yaml", idata=musicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

Saving data to newMusicDiscogsAlbumMap.yaml
Saved data to newMusicDiscogsAlbumMap.yaml
  --> This file is 1.6kB.
Found 9 music <-> discogs albums maps


In [99]:
musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

Loading data from newMusicDiscogsAlbumMap.yaml
  --> This file is 1.3kB.
Loading newMusicDiscogsAlbumMap.yaml
Found 5 music <-> discogs albums maps


## Move To Matched

In [100]:
from fsUtils import mkDir, isDir, moveDir
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        musicMap   = musicAlbumIDMap.get(artistName)
        if musicMap is None:
            continue
        for myAlbumName,albumVal in musicMap.items():
            matchedDir = setDir(dirval, "Match")
            mkDir(matchedDir)
            srcDir = setDir(dirval, myAlbumName)
            if not isDir(srcDir):
                print("{0} does not exist".format(srcDir))
                continue
            dstDir = setDir(matchedDir, discConv(myAlbumName))
            if isDir(dstDir):
                print("{0} already exists".format(dstDir))
                continue
                
            print("Moving {0}  --->  {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir)

/Users/tgadfort/matched/A
Moving /Users/tgadfort/matched/A/AC-DC/'74 Jailbreak  --->  /Users/tgadfort/matched/A/AC-DC/Match/'74 Jailbreak
Moving /Users/tgadfort/matched/A/AC-DC/Back In Black  --->  /Users/tgadfort/matched/A/AC-DC/Match/Back In Black
Moving /Users/tgadfort/matched/A/AC-DC/Ballbreaker  --->  /Users/tgadfort/matched/A/AC-DC/Match/Ballbreaker
Moving /Users/tgadfort/matched/A/AC-DC/Dirty Deeds Done Dirt Cheap  --->  /Users/tgadfort/matched/A/AC-DC/Match/Dirty Deeds Done Dirt Cheap
Moving /Users/tgadfort/matched/A/AC-DC/Dirty Deeds Done Dirt Cheap (Live)  --->  /Users/tgadfort/matched/A/AC-DC/Match/Dirty Deeds Done Dirt Cheap (Live)
Moving /Users/tgadfort/matched/A/AC-DC/Flick Of The Switch  --->  /Users/tgadfort/matched/A/AC-DC/Match/Flick Of The Switch
Moving /Users/tgadfort/matched/A/AC-DC/Fly On The Wall  --->  /Users/tgadfort/matched/A/AC-DC/Match/Fly On The Wall
Moving /Users/tgadfort/matched/A/AC-DC/For Those About To Rock (We Salute You)  --->  /Users/tgadfort/matche

# Move To Album Type

In [101]:
moveData = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):
        matchedDir = setDir(dirval, "Match")
        aTypes     = ['Albums', 'Singles & EPs', 'Compilations', 'Videos', 'Miscellaneous', 'Visual', 'DJ Mixes']
        albums     = [getDirBasics(x)[-1] for x in findDirs(matchedDir)]
        albums     = [x for x in albums if x not in aTypes]
        if len(albums) == 0:
            continue
        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        
        discogsArtist = myMusicNameIDMap[artistName]
        
        idx                      = discogsArtist[0]
        discogsArtistName        = discogsArtist[1]
        discogsArtistAlbumsData  = getRowByIndex(artistAlbumsDB, idx)
        discogsArtistAlbums      = discogsArtistAlbumsData["Albums"]
        
        
        for myAlbumName in albums:
            myAlbumName = normalize('NFC', myAlbumName)
            maxRatio    = None
            dbAlbumData = {}
            for albumType, albumTypeData in discogsArtistAlbums.items():
                for albumID, dbAlbumName in albumTypeData.items():
                    convDiscogsAlbumName = discConv(dbAlbumName)
                    s = SequenceMatcher(None, myAlbumName, convDiscogsAlbumName)
                    ratio = round(s.ratio(), 2)
                    if ratio > 0.0:
                        if maxRatio is None:
                            maxRatio = ratio
                        else:
                            if ratio < maxRatio:
                                continue
                            maxRatio = ratio

                        if moveData.get(artistName) is None:
                            print('  ',artistName)
                            moveData[artistName] = {"Dir": matchedDir, "Albums": {}}
                        moveData[artistName]["Albums"][myAlbumName] = [albumID, albumType, convDiscogsAlbumName]
       
            if maxRatio is not None:
                print('\t{0: <50}'.format(myAlbumName),moveData[artistName]["Albums"][myAlbumName])

/Users/tgadfort/matched/A
   AC-DC
	'74 Jailbreak                                      ['8460', 'Compilations', "'74 Jailbreak"]
	Back In Black                                      ['324463', 'Singles & EPs', 'Back In Black']
	Ballbreaker                                        ['3654907', 'Singles & EPs', 'Ballbreaker']
	Dirty Deeds Done Dirt Cheap                        ['469928', 'Singles & EPs', 'Dirty Deeds Done Dirt Cheap']
	Dirty Deeds Done Dirt Cheap (Live)                 ['201450', 'Singles & EPs', 'Dirty Deeds Done Dirt Cheap (Live)']
	Flick Of The Switch                                ['545065', 'Singles & EPs', 'Flick Of The Switch']
	Fly On The Wall                                    ['291661', 'Videos', 'Fly On The Wall']
	For Those About To Rock (We Salute You)            ['8663', 'Singles & EPs', 'For Those About To Rock (We Salute You)']
	High Voltage                                       ['8437', 'Albums', 'High Voltage']
	Highway To Hell                              

In [72]:
saveFile(ifile="matchedMoveMusicDiscogsAlbumMap.yaml", idata=moveData, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(moveData)))

Saving data to matchedMoveMusicDiscogsAlbumMap.yaml
Saved data to matchedMoveMusicDiscogsAlbumMap.yaml
  --> This file is 3.6kB.
Found 18 music <-> discogs albums maps


In [73]:
moveData = getFile(ifile="matchedMoveMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(moveData)))

Loading data from matchedMoveMusicDiscogsAlbumMap.yaml
  --> This file is 3.6kB.
Loading matchedMoveMusicDiscogsAlbumMap.yaml
Found 18 music <-> discogs albums maps


In [102]:
atypes = {}
for artistName,artistMoveData in moveData.items():
    dirval = artistMoveData["Dir"]
    albums = artistMoveData["Albums"]
    for myAlbumName, dbAlbumData in albums.items():
        albumID, albumType, convDiscogsAlbumName = dbAlbumData[0], dbAlbumData[1], dbAlbumData[2]
        atypes[albumType] = True
        albumTypeDir = setDir(dirval, albumType)
        mkDir(albumTypeDir)
        srcDir  = setDir(dirval, myAlbumName)
        dstName = " :: ".join([myAlbumName, "[{0}]".format(albumID)])
        dstDir  = setDir(albumTypeDir, dstName)
        print("Moving {0}".format(srcDir))
        moveDir(srcDir, dstDir)

Moving /Users/tgadfort/matched/A/AC-DC/Match/'74 Jailbreak
Moving /Users/tgadfort/matched/A/AC-DC/Match/Back In Black
Moving /Users/tgadfort/matched/A/AC-DC/Match/Ballbreaker
Moving /Users/tgadfort/matched/A/AC-DC/Match/Dirty Deeds Done Dirt Cheap
Moving /Users/tgadfort/matched/A/AC-DC/Match/Dirty Deeds Done Dirt Cheap (Live)
Moving /Users/tgadfort/matched/A/AC-DC/Match/Flick Of The Switch
Moving /Users/tgadfort/matched/A/AC-DC/Match/Fly On The Wall
Moving /Users/tgadfort/matched/A/AC-DC/Match/For Those About To Rock (We Salute You)
Moving /Users/tgadfort/matched/A/AC-DC/Match/High Voltage
Moving /Users/tgadfort/matched/A/AC-DC/Match/Highway To Hell
Moving /Users/tgadfort/matched/A/AC-DC/Match/Let There Be Rock
Moving /Users/tgadfort/matched/A/AC-DC/Match/Stiff Upper Lip
Moving /Users/tgadfort/matched/A/AC-DC/Match/The Razors Edge
Moving /Users/tgadfort/matched/A/AC-DC/Match/Who Made Who
Moving /Users/tgadfort/matched/C/Cyndi Lauper/Match/Shine
Moving /Users/tgadfort/matched/C/Cyndi La

In [ ]:
list(atypes.keys())

In [ ]:
for artistName,v in musicAlbumIDMap.items():
    print(artistName)
    for myAlbumName,v2 in v.items():
        print("\t",myAlbumName,'\t',v2)

In [ ]:
if True:
    saveFile(ifile="skipMusicDiscogsAlbumMap.yaml", idata=skipMusicAlbumIDMap, debug=True)
    print("Found {0} music <-> discogs albums maps".format(len(skipMusicAlbumIDMap)))
skipMusicAlbumIDMap

In [ ]:
saveFile(ifile="newMusicDiscogsAlbumMap.yaml", idata=musicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
musicAlbumIDMap = getFile(ifile="newMusicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

## Merge Maps (if needed)

In [ ]:
for artistName,v in musicAlbumIDMap.items():
    for myAlbumName,v2 in v.items():
        if myMusicAlbumIDMap.get(artistName) is None:
            print("Adding {0}".format(artistName))
            myMusicAlbumIDMap[artistName] = {}
        if myMusicAlbumIDMap[artistName].get(myAlbumName) is None:
            print("Adding {0}/{1} ---> {2}".format(artistName, myAlbumName, v2))
            myMusicAlbumIDMap[artistName][myAlbumName] = v2

In [ ]:
from ioUtils import saveFile
saveFile(ifile="musicDiscogsAlbumMap.p", idata=myMusicAlbumIDMap, debug=True)
print("Found {0} music <-> discogs albums maps".format(len(myMusicAlbumIDMap)))

In [ ]:
saveFile(ifile="musicDiscogsAlbumMap.yaml", idata=myMusicAlbumIDMap, debug=True)

# Rename Albums

In [ ]:
musicAlbumIDMap = getFile(ifile="musicDiscogsAlbumMap.yaml", debug=True)
print("Found {0} music <-> discogs albums maps".format(len(musicAlbumIDMap)))

In [ ]:
renames = {}
for dirN in findDirs("/Users/tgadfort/matched"):
    print(dirN)
    for dirval in findDirs(dirN):        
        artistName = getDirBasics(dirval)[-1]
        artistName = normalize('NFC', artistName)
        if musicAlbumIDMap.get(artistName) is None:
            continue
        print("    {0}".format(artistName))
        for album, albumdata in musicAlbumIDMap[artistName].items():
            print("\t[{0}]   ====>   [{1}]".format(album,albumdata[1]))
            srcAlbumName = album
            dstAlbumName = discConv(albumdata[1])
            if srcAlbumName != dstAlbumName:
                if renames.get(dirval) is None:
                    renames[dirval] = {}
                renames[dirval][srcAlbumName] = dstAlbumName

In [ ]:
saveFile(ifile="musicRenames.yaml", idata=renames, debug=True)

In [ ]:
renameData = getFile(ifile="musicRenames.yaml", debug=True)

In [ ]:
from fsUtils import isDir
for dirval,artistdata in renameData.items():
    for src,dst in artistdata.items():
        srcDir = setDir(dirval, src)
        dstDir = setDir(dirval, dst)
        print(dstDir,isDir(dstDir))
        if srcDir != dstDir and not isDir(dstDir):
            print("Moving {0} -> {1}".format(srcDir, dstDir))
            moveDir(srcDir, dstDir, debug=True)